In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from kerastuner.tuners import Hyperband
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# 假設有一組收盤價的時間序列數據 closing_prices
df=pd.read_csv('../Data/BTCUSD-all.csv')
closing_prices=df['Close']
# 創建時間窗口的訓練集數據
def create_dataset(closing_prices, time_window):
    X, y = [], []
    for i in range(len(closing_prices) - time_window):
        X.append(closing_prices[i:i+time_window])
        y.append(closing_prices[i+time_window])
    return np.array(X), np.array(y)

# 設置時間窗口大小
time_window = 7

# 創建訓練集和測試集
X, y = create_dataset(closing_prices, time_window)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 調整數據形狀以符合 LSTM 模型的要求
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# 定義超參數搜索空間
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                   return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 定義 EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0)

# 定義交叉驗證
kf = KFold(n_splits=5)

# 定義 Hyperband 調節器
tuner = Hyperband(build_model,
                  objective='val_loss',
                  max_epochs=100,
                  factor=3)

# 開始搜索最佳超參數
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]    
    tuner.search(X_train_fold, y_train_fold, epochs=10, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

# 獲取最佳模型和最佳超參數
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# 使用最佳模型預測
train_predictions = best_model.predict(X_train)
test_predictions = best_model.predict(X_test)

# 計算訓練集和測試集的 MAE
train_mae = mean_absolute_error(y_train, train_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)

print("訓練集的 MAE:", train_mae)
print("測試集的 MAE:", test_mae)

Trial 4 Complete [00h 00m 05s]
val_loss: 470836096.0

Best val_loss So Far: 470836096.0
Total elapsed time: 00h 00m 19s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
21/21 [==============================] - 0s 4ms/step
訓練集的 MAE: 14355.071553455438
測試集的 MAE: 13184.818218936776
最佳超參數: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x0000028E131B7F70>
